In [11]:
import pandas as pd
import numpy as np
import joblib
import sys

In [23]:
#np.set_printoptions(threshold=sys.maxsize)

In [3]:
# read in feature data
features3_csv = pd.read_csv("../FeatureExtracting/stage3_similarity_features.csv")

In [4]:
features3_csv

,Test,Reference,Similarity,jaccard_once_keyword,jaccard_multi_keyword,tf_cosine_appearance,tfidf_cosine_appearance,tf_cosine_times,tfidf_cosine_times,Sim_Hash_Similarity,Min_Similarity
0,100,1002,0.133333,0.1,0.071429,0.133333,0.038855,0.123603,0.047808,0.33,0.000000
1,100,1003,0.066667,0.0,0.052632,0.115470,0.019133,0.099177,0.018642,0.26,0.164062
2,100,101,0.133333,1.0,0.105263,0.210819,0.106684,0.364363,0.144300,0.15,0.132812
3,100,1013,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.32,0.000000
4,100,1017,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.35,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
175975,995,981,0.142857,0.0,0.045455,0.094491,0.022252,0.552917,0.210549,0.14,0.445312
175976,995,982,0.285714,0.1,0.100000,0.195180,0.100620,0.222871,0.072412,0.26,0.000000
175977,995,983,0.142857,0.0,0.100000,0.188982,0.055576,0.386444,0.129999,0.33,0.093750
175978,995,989,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.22,0.078125


In [5]:
# read S3 feature data
X3_similarity = features3_csv[['Similarity', 'jaccard_once_keyword', 'tf_cosine_appearance', 'tfidf_cosine_appearance']].to_numpy()
X3_appearance = np.load('../FeatureExtracting/stage3_appearance_feature.npy')


In [6]:
X3_appearance = np.delete(X3_appearance, 1529, 1)
X3_appearance = np.delete(X3_appearance, 764, 1)

In [8]:
# read in feature data
X3 = np.concatenate((X3_similarity, X3_appearance), axis=1)
X3.shape

(175980, 1532)

In [13]:
X3[0]

array([0.13333333, 0.1       , 0.13333333, 0.03885534, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 1.        , 1.        , 1.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 1.        , 1.        , 1.        , 0.        ,
       0.        , 1.        , 0.        , 0.        , 0.        ,
       0.        , 1.        , 1.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

In [14]:
# take only article pair orders
article_pairs = features3_csv[["Test", "Reference"]]
article_pairs

,Test,Reference
0,100,1002
1,100,1003
2,100,101
3,100,1013
4,100,1017
...,...,...
175975,995,981
175976,995,982
175977,995,983
175978,995,989


In [15]:
# turn X3 features to frame
X3_df = pd.DataFrame(X3)
X3_df

,0,1,2,3,4,5,6,7,8,9,...,1522,1523,1524,1525,1526,1527,1528,1529,1530,1531
0,0.133333,0.1,0.133333,0.038855,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.066667,0.0,0.115470,0.019133,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.133333,1.0,0.210819,0.106684,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175975,0.142857,0.0,0.094491,0.022252,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
175976,0.285714,0.1,0.195180,0.100620,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
175977,0.142857,0.0,0.188982,0.055576,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
175978,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
# join the article times & similarity times features with article pairs
joined_df = pd.merge(article_pairs, X3_df, left_index=True, right_index=True)
joined_df

,Test,Reference,0,1,2,3,4,5,6,7,...,1522,1523,1524,1525,1526,1527,1528,1529,1530,1531
0,100,1002,0.133333,0.1,0.133333,0.038855,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,100,1003,0.066667,0.0,0.115470,0.019133,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,100,101,0.133333,1.0,0.210819,0.106684,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,100,1013,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,100,1017,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175975,995,981,0.142857,0.0,0.094491,0.022252,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
175976,995,982,0.285714,0.1,0.195180,0.100620,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
175977,995,983,0.142857,0.0,0.188982,0.055576,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
175978,995,989,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
# read in the best results
xgb_df = pd.read_csv("csv_results/3_XGBOOST.csv")
xgb_df

,Test,Reference
0,101,487
1,101,496
2,101,502
3,101,53
4,101,61
...,...,...
967,995,842
968,995,900
969,995,914
970,995,920


In [18]:
# join back features from the best csv result
final_df = pd.merge(xgb_df, joined_df, on=["Test", "Reference"], how="left")
final_df

,Test,Reference,0,1,2,3,4,5,6,7,...,1522,1523,1524,1525,1526,1527,1528,1529,1530,1531
0,101,487,0.833333,0.000000,0.680414,0.641479,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,101,496,0.833333,0.000000,0.566139,0.508676,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,101,502,0.833333,0.000000,0.481125,0.440961,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,101,53,0.833333,0.000000,0.372678,0.319508,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,101,61,0.833333,0.000000,0.372678,0.319508,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
967,995,842,1.000000,0.071429,0.518875,0.390723,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
968,995,900,0.857143,1.000000,0.801784,0.770549,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
969,995,914,1.000000,1.000000,0.881917,0.830533,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
970,995,920,1.000000,2.000000,0.935414,0.914944,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# get the numpy features of the best results 
X3_bests = final_df.copy()
del X3_bests["Test"]
del X3_bests["Reference"]
X3_bests_np = X3_bests.to_numpy()
X3_bests_np

In [20]:
len(X3_bests_np[0])

1532

In [21]:
len(X3_bests_np)

972

In [22]:
# write to a numpy file
with open("X3_xgb3_lgbm8.npy", "wb") as f:
    np.save(f, X3_bests_np)